In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import PIL.Image
from sklearn.model_selection import train_test_split
import pathlib
import os

In [ ]:
from google.colab import drive , files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def create_multi_organ_model(img_height, img_width, num_classes):
    """Create a model for multi-organ classification"""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
def preprocess_images(image_files, img_height, img_width):
    """Preprocess images with error handling"""
    processed_images = []
    for img_path in image_files:
        try:
            img = PIL.Image.open(img_path).convert('RGB')
            img = img.resize((img_width, img_height))
            img_array = np.array(img, dtype=np.float32) / 255.0
            processed_images.append(img_array)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue

    if not processed_images:
        raise ValueError(f"No valid images were processed")

    return np.array(processed_images)

In [ ]:
def prepare_data_for_training(folder_paths, labels, img_height, img_width):
    """Prepare data from multiple folders with corresponding labels"""
    all_images = []
    all_labels = []

    # Verify we have valid input
    if not folder_paths or not labels:
        raise ValueError("Empty folder paths or labels provided")

    if len(folder_paths) != len(labels):
        raise ValueError(f"Number of folder paths ({len(folder_paths)}) doesn't match number of labels ({len(labels)})")

    num_classes = max(labels) + 1
    print(f"Number of classes detected: {num_classes}")

    for folder_path, label in zip(folder_paths, labels):
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist, skipping...")
            continue

        data_dir = pathlib.Path(folder_path)
        image_files = list(data_dir.glob('*.jpg'))
        image_files.extend(data_dir.glob('*.png'))
        if not image_files:
            print(f"Warning: No images found in {folder_path}")
            continue

        print(f"Processing folder {folder_path} with label {label}")
        images = preprocess_images(image_files, img_height, img_width)
        labels_array = np.full(len(images), label)

        all_images.append(images)
        all_labels.append(labels_array)

    if not all_images:
        raise ValueError("No valid images found in any of the folders")

    X = np.concatenate(all_images, axis=0)
    y = np.concatenate(all_labels, axis=0)

    print(f"Shape of X before split: {X.shape}")
    print(f"Shape of y before split: {y.shape}")
    print(f"Unique labels found: {np.unique(y)}")

    # Convert to categorical with explicit num_classes
    y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    print(f"Shape of y after categorical encoding: {y.shape}")

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def train_model(folder_paths, organ_labels, img_height=128, img_width=128, batch_size=32, epochs=10):
    """Train the multi-organ classifier"""
    print("Starting model training...")
    print(f"Number of folders: {len(folder_paths)}")
    print(f"Labels: {organ_labels}")

    # Prepare data
    X_train, X_val, y_train, y_val = prepare_data_for_training(
        folder_paths, organ_labels, img_height, img_width
    )

    # Create model with correct number of classes
    num_classes = y_train.shape[1]
    print(f"Creating model with {num_classes} output classes")
    model = create_multi_organ_model(img_height, img_width, num_classes)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    return model, history

In [ ]:
if __name__ == "__main__":
    folder_paths = [
        "/content/drive/My Drive/dataset/brain",
        "/content/drive/My Drive/dataset/lung",

    ]

    organ_labels = [0 , 1 , 2 , 3]

    try:
        model, history = train_model(folder_paths, organ_labels, epochs=1)
        model.save('/content/drive/My Drive/multi_organ_classifier.keras')
        print("Model training completed and saved successfully!")
    except Exception as e:
        print(f"Error during training: {str(e)}")

Starting model training...
Number of folders: 4
Labels: [0, 1, 2, 3]
Number of classes detected: 4
Processing folder /content/drive/My Drive/dataset/brain with label 0
Processing folder /content/drive/My Drive/dataset/lung with label 1
Processing folder /content/drive/My Drive/dataset/kidney with label 2
Processing folder /content/drive/My Drive/dataset/hand with label 3
Shape of X before split: (6645, 128, 128, 3)
Shape of y before split: (6645,)
Unique labels found: [0 1 2 3]
Shape of y after categorical encoding: (6645, 4)
Creating model with 4 output classes
167/167 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.8511 - loss: 0.4210 - val_accuracy: 0.9940 - val_loss: 0.0336
Model training completed and saved successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')